In [1]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use("seaborn")

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch dataset


In [2]:
from SCFInitialGuess.utilities.dataset import make_butadien_dataset


dataset, molecules = make_butadien_dataset(
    np.load("molecules_platin_lanl2dz.npy"),
    *np.load("dataset_platin_lanl2dz.npy")
)

[-] 2018-05-23 14:41:53: Data set normalized. Mean value std: 0.007695333664764158


# Extract diagonal 

In [3]:
def extract_diagonal(x, dim=88):
    x_batch = x.reshape(-1, dim, dim)
    return np.array(list(map(np.diag, x_batch))).reshape(-1, 88)


dataset.training = (dataset.training[0], extract_diagonal(dataset.training[1]))
dataset.validation = (dataset.validation[0], extract_diagonal(dataset.validation[1]))
dataset.testing = (dataset.testing[0], extract_diagonal(dataset.testing[1]))

# Train a network

In [5]:
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.cost_functions import RegularizedMSE, MSE
from SCFInitialGuess.nn.training import Trainer

dim = 88

trainer = Trainer(
    EluTrNNN([dim**2, 400, 400, 200, 150, dim]),
    cost_function=MSE(),#RegularizedMSE(alpha=1e-7),
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3)
)

trainer.setup()

network, sess = trainer.train(
    dataset,
    convergence_threshold=5e-7
)
    

NameError: name 'MSE' is not defined